In [1]:
from dotenv import load_dotenv
load_dotenv('/Users/mdmehranabul/Desktop/Learning/Langchain_Ollama/.env')

True

### Simple Chain

In [2]:
from langchain_core.prompts import (SystemMessagePromptTemplate, 
                                    HumanMessagePromptTemplate,
                                    PromptTemplate,
                                    ChatMessagePromptTemplate,
                                    ChatPromptTemplate)
from langchain_ollama import ChatOllama
from langchain_core.output_parsers  import StrOutputParser

base_url="http://localhost:11434"
model='llama3.2:3b'


llm=ChatOllama(base_url=base_url,model=model)
llm

ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434')

In [3]:
template=ChatPromptTemplate.from_template("{prompt}")
chain=template | llm | StrOutputParser()

about="My name is Mehran. I am a data scientist and AI enthusiast. I have worked at various companies like TCS and Amazon"
chain.invoke({'prompt':about})

"Nice to meet you, Mehran! It's great to hear that you're a data scientist and AI enthusiast with experience working at reputable companies like TCS and Amazon.\n\nAs a data scientist, you must have had the opportunity to work on a wide range of projects, from predictive modeling to machine learning algorithms. I'm sure your expertise in AI has helped drive business value for the companies you've worked with.\n\nWhat specific areas of AI or data science are you most interested in? Are you working on any exciting projects or exploring new technologies like deep learning or natural language processing?\n\n(Also, feel free to share more about your background and experiences if you'd like!)"

In [4]:
prompt="What is my name?"
chain.invoke({'prompt':prompt})

"I don't have any information about you, so I don't know your name. This conversation just started, and we haven't exchanged any personal details yet. Would you like to tell me your name?"

### Runnable with message history

In [5]:
from langchain_core.prompts import (SystemMessagePromptTemplate, 
                                    HumanMessagePromptTemplate,
                                    PromptTemplate,
                                    ChatMessagePromptTemplate,
                                    ChatPromptTemplate)
from langchain_ollama import ChatOllama
from langchain_core.output_parsers  import StrOutputParser
from langchain_core.messages import HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [6]:
def get_session_history(session_id):
    return SQLChatMessageHistory(session_id,"sqlite:///chat_history.db")

In [7]:
runnable_with_history = RunnableWithMessageHistory(chain, get_session_history)

In [18]:
user_id='mehran_123'
history=get_session_history(user_id)

history.get_messages()

[]

In [17]:
history.clear()

In [19]:
about

'My name is Mehran. I am a data scientist and AI enthusiast. I have worked at various companies like TCS and Amazon'

In [20]:
runnable_with_history.invoke([HumanMessage(content=about)],config={'configurable':{'session_id':user_id}})

"Hello Mehran, \n\nIt's great to hear about your background in data science and AI enthusiasm! Working with well-known companies such as TCS and Amazon must have provided you with a broad range of experiences. What specific areas or projects within these fields are you most passionate about? Are there any new technologies or advancements that you're particularly interested in?\n\nFeel free to share more about your interests, goals, or any current challenges you're facing in your work."

In [22]:
runnable_with_history.invoke([HumanMessage(content="What is my name?")],config={'configurable':{'session_id':user_id}})

'This appears to be a sample conversation between a human user (Mehran) and a conversational AI system. The conversation starts with Mehran introducing himself as a data scientist and AI enthusiast, followed by the AI system responding with a friendly message asking about his interests and areas of passion.\n\nThe next message from Mehran is a simple question asking "What is my name?", which seems to be an attempt to test the AI system\'s understanding or provide some context for the conversation.\n\nThe AI system responds by analyzing Mehran\'s previous messages and provides a summary of his background, interests, and current challenges in his work. It then suggests potential follow-up questions that could help deepen the conversation and establish a meaningful connection with Mehran.\n\nHowever, Mehran repeats the question "What is my name?" again, indicating that he may be looking for a more personalized response or trying to see how the AI system reacts to this specific question.\n

### Message history with Dictionary like Inputs

In [23]:
from langchain_core.prompts import (SystemMessagePromptTemplate, 
                                    HumanMessagePromptTemplate,
                                    PromptTemplate,
                                    ChatMessagePromptTemplate,
                                    ChatPromptTemplate,
                                    MessagesPlaceholder)
from langchain_ollama import ChatOllama
from langchain_core.output_parsers  import StrOutputParser
from langchain_core.messages import HumanMessage

In [24]:
system=SystemMessagePromptTemplate.from_template("You are a helpful assistant.")
human=HumanMessagePromptTemplate.from_template("{input}")

messages=[system,MessagesPlaceholder(variable_name='history'),human]

prompt=ChatPromptTemplate.from_messages(messages=messages)

chain=prompt | llm | StrOutputParser()


runnable_with_history= RunnableWithMessageHistory(chain, get_session_history,input_messages_key='input',history_messages_key='history')

In [25]:
def chat_with_llm(session_id,input):
    output=runnable_with_history.invoke({'input':input},config={'configurable':{'session_id':session_id}})
    return output

In [26]:
user_id='mehran_001'
chat_with_llm(user_id,about)

"Nice to meet you, Mehran! It's great to hear that you're a data scientist and AI enthusiast with a rich background working at renowned companies like TCS and Amazon.\n\nAs a data scientist, you must have expertise in leveraging data analytics and machine learning techniques to drive business value. Your experience at these top companies suggests that you've had the opportunity to work on various projects and contribute to innovative solutions.\n\nWhat specific areas of AI or data science are you most passionate about? Are you working on any exciting projects currently, or is there a particular problem you're trying to solve in your research or professional endeavors?\n\nFeel free to share more about yourself, Mehran! I'm all ears."

In [27]:
chat_with_llm(user_id,"What is my name?")

'Your name is Mehran.'